In [1]:
from z3 import Solver, And
from meyer.meyer import conclude
from meyer.loop import fix_rep, wloop, LOOP_NUM
from meyer.invariant import is_ivr_of, is_livr_of
from meyer.program import prog, progs
from meyer.basic_constructs import Choi
from meyer.concurrency import NAtom
from meyer.conditionals import Ite, GCond
from meyer.equivalence import eq_ran, eq_pre
from meyer.util.z3py_set import set, sets
from meyer.util.z3py_rel import well_founded
s = Solver()

## P63
CorestとCompの順番を変えても成り立つ<br>
P63が成り立つには P11とP27が成り立つ必要があるはずなのに…<br>
LOOP_NUMは20までは成り立つことを確認している<br>
Z3で帰納法を表現する方法を調べよう

In [2]:
title = "P63 The loop l = from a until C loop b end can be writen ∪(i≧0)q_i where q_i is a;(C':b)^i＼C"
a,b = progs(s, 'a b')
C = set('C')
rhs = Choi(*[(a ^ fix_rep(b / -C, i)) // C for i in range(LOOP_NUM)])
conclude(s, wloop(a, C, b) == rhs, title)

P63 The loop l = from a until C loop b end can be writen ∪(i≧0)q_i where q_i is a;(C':b)^i＼C
Universe = U, has 3 element(s)
Holds: unsat 



∪ran(q_i) = ran(∪q_i) (∵ ran(r)∪ran(s) = ran(r∪s))

In [3]:
title = "P63 As a consequence, ran(l) = ∪ran(q_i)"
a,b = progs(s, 'a b')
C = set('C')
rhs = Choi(*[a ^ fix_rep(b / -C, i) // C for i in range(LOOP_NUM)])
conclude(s, eq_ran(wloop(a, C, b), rhs), title)

P63 As a consequence, ran(l) = ∪ran(q_i)
Universe = U, has 3 element(s)
Holds: unsat 



In [4]:
title = "P64 Any I disjoint from p's domain is an invaliant of p"
I = set('I')
p = prog(s, 'p')
s.add(I != p.dom())
conclude(s, is_ivr_of(I, p), title)

P64 Any I disjoint from p's domain is an invaliant of p
Universe = U, has 3 element(s)
Holds: unsat 



In [5]:
title = "P65 If I and J are invariants of p, so are I∪J and I∩J"
I, J = sets('I J')
p = prog(s, 'p')
s.add(is_ivr_of(I, p), is_ivr_of(J, p))
conclude(s, And(is_ivr_of(I | J, p), is_ivr_of(I & J, p)), title)

P65 If I and J are invariants of p, so are I∪J and I∩J
Universe = U, has 3 element(s)
Holds: unsat 



In [6]:
title = "P66 If I is an invariant of p1 and p2 ⊆ p1, then I is an invariant of p2/Pre1"
p1, p2 = progs(s, "p1 p2")
I = set('I')
s.add(is_ivr_of(I, p1), p2 <= p1)
conclude(s, is_ivr_of(I, p2 / p1.pre()), title)

P66 If I is an invariant of p1 and p2 ⊆ p1, then I is an invariant of p2/Pre1
Universe = U, has 3 element(s)
Holds: unsat 



In [7]:
title = "P67 All the program operators defined so far are invariant-preserving: Choice"
I = set('I')
p1, p2 = progs(s, 'p1 p2')
s.add(eq_pre(p1, p2)) # This makes hold but too strong
s.add(is_ivr_of(I, p1), is_ivr_of(I, p2))
conclude(s, is_ivr_of(I, p1 | p2), title)

P67 All the program operators defined so far are invariant-preserving: Choice
Universe = U, has 3 element(s)
Holds: unsat 



In [8]:
title = "P67 All the program operators defined so far are invariant-preserving: Composition"
I = set('I')
p1, p2 = progs(s, 'p1 p2')
s.add(is_ivr_of(I, p1), is_ivr_of(I, p2))
conclude(s, And(is_ivr_of(I, p1 ^ p2), is_ivr_of(I, p2 ^ p1)), title)

P67 All the program operators defined so far are invariant-preserving: Composition
Universe = U, has 3 element(s)
Holds: unsat 



In [9]:
title = "P67 All the program operators defined so far are invariant-preserving: Restriction"
C, I = sets('C I')
p = prog(s, 'p')
s.add(is_ivr_of(I, p))
conclude(s, is_ivr_of(I, p / C), title)

P67 All the program operators defined so far are invariant-preserving: Restriction
Universe = U, has 3 element(s)
Holds: unsat 



In [10]:
title = "P67 All the program operators defined so far are invariant-preserving: Corestriction"
C, I = sets('C I')
p = prog(s, 'p')
s.add(is_ivr_of(I, p))
conclude(s, is_ivr_of(I, p // C), title)

P67 All the program operators defined so far are invariant-preserving: Corestriction
Universe = U, has 3 element(s)
Holds: unsat 



In [11]:
title = "P67 All the program operators defined so far are invariant-preserving: Atomic Concurrency"
I = set('I')
p1, p2 = progs(s, 'p1 p2')
s.add(eq_pre(p1, p2)) # This makes hold but too strong
s.add(is_ivr_of(I, p1), is_ivr_of(I, p2))
conclude(s, is_ivr_of(I, p1 & p2), title)

P67 All the program operators defined so far are invariant-preserving: Atomic Concurrency
Universe = U, has 3 element(s)
Holds: unsat 



In [12]:
title = "P67 All the program operators defined so far are invariant-preserving: Atomic Concurrency"
I = set('I')
p1, p2, q = progs(s, 'p1 p2 q')
s.add(eq_pre(p1, p2), eq_pre(p2, q)) # This makes hold but too strong
s.add(is_ivr_of(I, p1), is_ivr_of(I, p2), is_ivr_of(I, q))
conclude(s, is_ivr_of(I, NAtom(p1, p2, q)), title)

P67 All the program operators defined so far are invariant-preserving: Atomic Concurrency
Universe = U, has 3 element(s)
Holds: unsat 



In [13]:
title = "P67 All the program operators defined so far are invariant-preserving: If then else"
C, I = sets('C I')
p1, p2 = progs(s, 'p1 p2')
s.add(is_ivr_of(I, p1), is_ivr_of(I, p2))
conclude(s, is_ivr_of(I, Ite(C, p1, p2)), title)

P67 All the program operators defined so far are invariant-preserving: If then else
Universe = U, has 3 element(s)
Holds: unsat 



In [14]:
title = "P67 All the program operators defined so far are invariant-preserving: Guarded conditional"
C1, C2, I = sets('C1 C2 I')
p1, p2 = progs(s, 'p1 p2')
s.add(eq_pre(p1, p2), eq_pre(p2, q)) # This makes hold but too strong
s.add(is_ivr_of(I, p1), is_ivr_of(I, p2))
conclude(s, is_ivr_of(I, GCond(C1, p1, C2, p2)), title)

P67 All the program operators defined so far are invariant-preserving: Guarded conditional
Universe = U, has 3 element(s)
Holds: unsat 



In [15]:
title = "P68 If I is a loop invariant of the loop l = (from a until C loop b end), then range of l is subset of C ∩ I"
C, I = sets('C I')
a, b = progs(s, 'a b')
l =  wloop(a, C, b)
s.add(is_ivr_of(I, l))
conclude(s, l.ran <= C & I)

Universe = U, has 3 element(s)
Holds: unsat 



In [16]:
title = "P69 For feasible a and b, the while loop is feasible if both '(domain of b) ∪ C is a loop variant' and 'C':post_b is well-founded'"
C = set('C')
a, b = progs(s, 'a b')
l = wloop(a, C, b)
s.add(+a, +b)
s.add(is_livr_of(C | b.dom(), a, C, b))
s.add(well_founded(b.post() / -C))
conclude(s, +l, title)

P69 For feasible a and b, the while loop is feasible if both '(domain of b) ∪ C is a loop variant' and 'C':post_b is well-founded'
Universe = U, has 3 element(s)
Holds: unsat 

